<a href="https://colab.research.google.com/github/wilberquito/AMLProject/blob/main/notebooks/EfficientNetB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientNetB4

In [ ]:
import zipfile
from pathlib import Path
import torch
import matplotlib.pyplot as plt

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    ! pip install torchvision
    ! pip install torchinfo
except:
    IN_COLAB = False

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    !rm -rf data
    data_path = Path('/content/drive/MyDrive/AML/dataset_CIFAR10.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall("data")

    !rm -rf modular
    data_path = Path('/content/drive/MyDrive/AML/modular.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall(".")

    !rm -rf trained
    !cp -rf /content/drive/MyDrive/AML/trained ./trained

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Show device available

In [ ]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'

## Show model arquitecture

In [ ]:
from torchinfo import summary
import modular.models as models
from prettytable import PrettyTable

In [ ]:
amlefficientnetb6 = models.AMLEfficientNetB6(out_dim=10)

In [ ]:
batch_size = 16
img_size = 528
summary(model=amlefficientnetb6, 
        input_size=(32, 3, img_size, img_size),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
AMLEfficientNetB6 (AMLEfficientNetB6)                             [32, 3, 528, 528]    [32, 10]             --                   Partial
├─EfficientNet (net)                                              [32, 3, 528, 528]    [32, 2304]           --                   False
│    └─Sequential (features)                                      [32, 3, 528, 528]    [32, 2304, 17, 17]   --                   False
│    │    └─Conv2dNormActivation (0)                              [32, 3, 528, 528]    [32, 56, 264, 264]   (1,624)              False
│    │    └─Sequential (1)                                        [32, 56, 264, 264]   [32, 32, 264, 264]   (8,094)              False
│    │    └─Sequential (2)                                        [32, 32, 264, 264]   [32, 40, 132, 132]   (156,922)            False
│    │    └─Sequential (3)                       

In [ ]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
amlefficientnetb6.unfreeze_base()
_ = count_parameters(amlefficientnetb6)

+--------------------------------------+------------+
|               Modules                | Parameters |
+--------------------------------------+------------+
|       net.features.0.0.weight        |    1512    |
|       net.features.0.1.weight        |     56     |
|        net.features.0.1.bias         |     56     |
|  net.features.1.0.block.0.0.weight   |    504     |
|  net.features.1.0.block.0.1.weight   |     56     |
|   net.features.1.0.block.0.1.bias    |     56     |
| net.features.1.0.block.1.fc1.weight  |    784     |
|  net.features.1.0.block.1.fc1.bias   |     14     |
| net.features.1.0.block.1.fc2.weight  |    784     |
|  net.features.1.0.block.1.fc2.bias   |     56     |
|  net.features.1.0.block.2.0.weight   |    1792    |
|  net.features.1.0.block.2.1.weight   |     32     |
|   net.features.1.0.block.2.1.bias    |     32     |
|  net.features.1.1.block.0.0.weight   |    288     |
|  net.features.1.1.block.0.1.weight   |     32     |
|   net.features.1.1.block.0

## Fine tunning

In [ ]:
import torch
import modular.datasets as datasets
import modular.models as models 
from pathlib import Path
from modular.engine import train
import torchvision.transforms as transforms
from modular.utils import set_seeds

set_seeds(seed=42)

# Model
amlefficientnetb6 = models.AMLEfficientNetB6(out_dim=10)
# Let base train
amlefficientnetb6.unfreeze_base()

train_transforms, validate_transforms = amlefficientnetb6.transforms, amlefficientnetb6.transforms

augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomPerspective(),
])

# train_transforms = transforms.Compose([augmentation, train_transforms])

# Optimizer
lr = 0.001
optimizer = torch.optim.Adam(params=amlefficientnetb6.parameters(),
                             lr=lr)

# Criterion
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs to train the model
epochs = 2

# Default device to train model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Where the model is saved
save_as = Path('finetune.amlefficientnetb6.pth')

# Mini batch
batch_size=8

train_dataloader = datasets.get_dataloader(folder_root='data/train',
                                           transformer=train_transforms,
                                           batch_size=batch_size,
                                           suffle=True)
validate_dataloader = datasets.get_dataloader(folder_root='data/validation',
                                           transformer=validate_transforms,
                                           batch_size=batch_size,
                                           suffle=False)

In [ ]:
results = train(model=amlefficientnetb6,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                save_as=save_as)

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

OutOfMemoryError: ignored

In [ ]:
from modular.utils import plot_loss_curves
plot_loss_curves(results)

In [ ]:
if IN_COLAB:
  from google.colab import files
  files.download('finetune.amlresnet101.pth')

## Training find tunning AMLResnet101

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    ! pip install torchvision
    ! pip install torchinfo
except:
    IN_COLAB = False

In [ ]:
import torch
from pathlib import Path

saved_data = torch.load(Path('trained/finetune.amlresnet101.pth'))
saved_data.keys()

In [ ]:
import torch
import modular.datasets as datasets
import modular.models as models 
from pathlib import Path
from modular.engine import train
import torchvision.transforms as transforms
from modular.utils import set_seeds

set_seeds(seed=42)

# Model
amlresnet101 = models.AMLResnet101(out_dim=10)
# Recover state dict
state_dict = saved_data['model']
# Load saved state dict into the model
amlresnet101.load_state_dict(state_dict)
# Freeze base
amlresnet101.freeze_base()

train_transforms, validate_transforms = amlresnet101.transforms, amlresnet101.transforms

augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.RandomPerspective(),
])

train_transforms = transforms.Compose([augmentation, train_transforms])

# Optimizer
lr = 0.001
optimizer = torch.optim.Adam(params=amlresnet101.parameters(),
                             lr=lr)

# Criterion
criterion = torch.nn.CrossEntropyLoss()

# Number of epochs to train the model
epochs = 10

# Default device to train model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Where the model is saved
save_as = Path('amlresnet101.pth')

# Mini batch
batch_size=750

train_dataloader = datasets.get_dataloader(folder_root='data/train',
                                           transformer=train_transforms,
                                           batch_size=batch_size,
                                           suffle=True)
validate_dataloader = datasets.get_dataloader(folder_root='data/validation',
                                           transformer=validate_transforms,
                                           batch_size=batch_size,
                                           suffle=False)

In [ ]:
results = train(model=amlresnet101,
                train_dataloader=train_dataloader,
                test_dataloader=validate_dataloader,
                optimizer=optimizer,
                criterion=criterion,
                epochs=epochs,
                device=device,
                save_as=save_as)

In [ ]:
from modular.utils import plot_loss_curves
plot_loss_curves(results)

In [ ]:
if IN_COLAB:
  from google.colab import files
  files.download('amlresnet101.pth')